* С помощью модуля pandas преобразуйте файл из .xlsx в .csv формат файл ./src/online_retail.xlsx

In [32]:
# Импортирование необходимой библиотеки
import pandas as pd 
# Чтение файла XLSX с помощью функции read_excel().
# Указать путь к файлу, имя листа (если их несколько) и другие параметры.
df = pd.read_excel('./src/online_retail.xlsx', sheet_name='Online Retail', header=0)
# Запись созданного объекта DataFrame в файл CSV с помощью функции to_csv()
df.to_csv('./src/online_retail.csv', index=False)

In [ ]:
# Импортирование необходимых библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop
from pyspark.sql.functions import when, array_contains, filter
from pyspark.sql.functions import col, lit, asc, desc, round, current_date, datediff

In [ ]:
# Инициализация Spark-сессии
spark = SparkSession.builder \
    .master("local") \
    .appName("onlineretail") \
    .getOrCreate()

In [ ]:
# Создание dataframe из скачанного файла
sparkDF_orig = spark.createDataFrame(df) 
sparkDF_orig.printSchema()
sparkDF_orig.show(3)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



24/07/07 20:00:53 WARN TaskSetManager: Stage 72 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows



24/07/07 20:00:57 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 72 (TID 43): Attempting to kill Python Worker


Подсчитайте следующие показатели:

    -Количество строк в файле

    -Количество уникальных клиентов

    -В какой стране совершается большинство покупок
    
    -Даты самой ранней и самой последней покупки на платформе

In [ ]:
# Чтобы посчитать количество строк в dataframe на PySpark, можно использовать функцию count():
line_count = sparkDF_orig.count()
print("Количество строк:", line_count)

Py4JJavaError: An error occurred while calling o266.count.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:75)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.C

In [ ]:
# Предвариьная обработка датафрейма
# Удаление из датафрейма строк, которые не содержат значения в столбце CustomerID
sparkDF_NotNull = sparkDF_orig.dropna(subset="CustomerID")

# Удаление из датафрейма строк, которые содержат некорректные значения стоимости и количества товара
sparkDF = sparkDF_NotNull.filter((sparkDF_NotNull.Quantity > 0) & (sparkDF_NotNull.UnitPrice > 0)).orderBy("CustomerID")
print("Количество строк после удаления ненужных: ", sparkDF.count())
# sparkDF.show(3)

RuntimeError: SparkContext or SparkSession should be created first.

In [ ]:
#Количество уникальных клиентов
countUniqCustomerID = sparkDF.select(countDistinct("CustomerID"))
countUniqCustomerID.show()

24/07/07 20:01:08 WARN TaskSetManager: Stage 80 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+
|count(DISTINCT CustomerID)|
+--------------------------+
|                      4338|
+--------------------------+



In [ ]:
# В какой стране совершается большинство покупок
# Группировка по стране (колонке "Country") и подсчёт количества строк по каждой стране
dfGroupByCountry = sparkDF.groupBy('Country').count()\
    .select(col("Country"), col("count").alias("countCountry"))
# dfGroupByCountry.show(3)
# сортировка полученного датафрэйма по убыванию по количеству покупок в каждой стране.
orderedDF = dfGroupByCountry.orderBy(col("count").desc())
# вывод только первой строки, в которой указана страна, в которой совершается большинство покупок
orderedDF.show(1)

24/07/07 20:01:12 WARN TaskSetManager: Stage 86 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+--------------+------------+
|       Country|countCountry|
+--------------+------------+
|United Kingdom|      354321|
+--------------+------------+
only showing top 1 row



In [ ]:
# Даты самой ранней и самой последней покупки на платформе
# строка с самой ранней покупкой на платформе
dateDFmin = sparkDF.orderBy(col("InvoiceDate").asc()).limit(1)
#dateDFmin.show(1)
# дата самой ранней покупки на платформе
dateMin = sparkDF.select(min(sparkDF.InvoiceDate))
dateMin.show()

# строка с самой последней покупкой на платформе
dateDFmax = sparkDF.orderBy(col("InvoiceDate").desc()).limit(1)
#dateDFmax.show()
# дата самой последней покупки на платформе
dateMax = sparkDF.select(max(sparkDF.InvoiceDate))
dateMax.show()

24/07/07 20:01:15 WARN TaskSetManager: Stage 89 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+
|   min(InvoiceDate)|
+-------------------+
|2010-12-01 08:26:00|
+-------------------+



24/07/07 20:01:18 WARN TaskSetManager: Stage 92 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+
|   max(InvoiceDate)|
+-------------------+
|2011-12-09 12:50:00|
+-------------------+



Проведите RFM-анализ клиентов платформы. 
Что такое RFM-анализ? 
  Обычно RFM-анализ используется в маркетинге для оценки ценности клиента на основе его:
  * Recency - Давность: как давно каждый покупатель совершил покупку?
    
  * Frequency- Частота: Как часто они что-то покупали?
    
  * Monetary - Денежная ценность: сколько денег они в среднем тратят при совершении покупок?

Добавьте в dataframe для каждого клиента 3 новых поля

In [ ]:
# Добавление колонки с общей стоимостью товара (цена*количество) 
# На основе этой колонки будет рассчитываться показатель Monetary
dfWithTotalCost = sparkDF.withColumn("TotalCost", sparkDF.UnitPrice * sparkDF.Quantity)\
    .withColumn("CurrentDate", current_date())
# Добаваление колонки, содержащей количество дней между текущей датой и датой самой поздней покупки
dfWithTotalCostAndDateDif = dfWithTotalCost.withColumn("DateDifference", datediff(dfWithTotalCost.CurrentDate, dfWithTotalCost.InvoiceDate))
dfWithTotalCostAndDateDif.show(2)

24/07/07 20:01:21 WARN TaskSetManager: Stage 95 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+-----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|         TotalCost|CurrentDate|DateDifference|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+-----------+--------------+
|   541431|    23166|MEDIUM CERAMIC TO...|   74215|2011-01-18 10:01:00|     1.04|   12346.0|United Kingdom|           77183.6| 2024-07-07|          4919|
|   537626|    85116|BLACK CANDELABRA ...|      12|2010-12-07 14:57:00|      2.1|   12347.0|       Iceland|25.200000000000003| 2024-07-07|          4961|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+-----------+--------------+
only showing top 2 rows



Для распределения клиентов по группам (ABC) выбираем следующий критерий:

1) Если значение показателя для клиента лучше среднего значения данного показателя и входит в интервал от 10 процентов разницы (по абсолютному значению, т.е. по модулю) лучшего и среднего значений до лучшего значения, 
ТО определеяем клиента в группу "A".

2) Если значение показателя для клиента лучше среднего значения и входит в интервал от среднего значения до 10 процентов разницы (по модулю) лучшего и среднего значений, 
ЛИБО
если значение показателя для клиента хуже среднего значения и входит в интервал от 60 процентов разницы (по модулю) худшего и среднего значения до среднего значения,
ТО определеяем клиента в группу "B".

3) Если значение показателя для клиента хуже среднего значения и входит в интервал от худшего значения до 60 процентов разницы (по модулю) худшего и среднего значений,
ТО определеяем клиента в группу "C".

In [ ]:
# Расчёт показателя Recency (Давность) на основе поиска максимального значения даты совершения покупки клиентом
dfRecency = dfWithTotalCostAndDateDif.groupBy('CustomerID')\
    .agg(max('DateDifference').alias('Recency')).orderBy("CustomerID")

# Определение максимального, минимального и среднего значения Recency (позволит приближенно оценить распределение признака)
dfRecencyMaxMin = dfRecency.select(max("Recency").alias("maxRec"),\
                                   min("Recency").alias("minRec"),\
                                   round(avg("Recency")).alias("avgRec"))   # функция round - для округления
maxRec = dfRecencyMaxMin.collect()[0].__getitem__('maxRec')
minRec = dfRecencyMaxMin.collect()[0].__getitem__('minRec')
avgRec = dfRecencyMaxMin.collect()[0].__getitem__('avgRec')
# Добавление колонки для указания группы покупателей по показателю Recency (Давность)
dfRecencyGroupABC = dfRecency\
    .withColumn("RecencyGroup",\
        when((dfRecency.Recency < avgRec) \
                & ((avgRec - dfRecency.Recency) <= (avgRec - minRec)*0.9)\
            , "A")\
        .when((dfRecency.Recency < avgRec) \
                | ((dfRecency.Recency >= avgRec)\
                    & ((dfRecency.Recency - avgRec) <= (maxRec - avgRec)*0.6))\
            , "B")\
        .otherwise("C"))

# Расчёт показателя Frequency на основе количества транзакций, совершённых клиентом
dfFrequency = dfWithTotalCostAndDateDif.groupBy("CustomerID").count()\
    .withColumnRenamed('count', 'Frequency')\
    .orderBy("CustomerID")
# Определение максимального, минимального и среднего значения Frequency (позволит приближенно оценить распределение признака)
dfFrequencyMaxMin = dfFrequency.select(max("Frequency").alias("maxFreq"),\
                                       min("Frequency").alias("minFreq"),\
                                       round(avg("Frequency")).alias("avgFreq"))
maxFreq = dfFrequencyMaxMin.collect()[0].__getitem__('maxFreq')
minFreq = dfFrequencyMaxMin.collect()[0].__getitem__('minFreq')
avgFreq = dfFrequencyMaxMin.collect()[0].__getitem__('avgFreq')
# Добавление колонки для указания группы покупателей по показателю Frequency (Частота покупок)
dfFrequencyGroupABC = dfFrequency\
    .withColumn("FrequencyGroup",\
        when((dfFrequency.Frequency > avgFreq)\
              & ((dfFrequency.Frequency - avgFreq) >= (maxFreq - avgFreq)*0.1)\
              , "A")\
        .when((dfFrequency.Frequency > avgFreq)\
                | ((dfFrequency.Frequency <= avgFreq)\
                    & ((avgFreq - dfFrequency.Frequency) >= (avgFreq - minFreq)*0.6))\
            , "B")\
        .otherwise("C"))

# Расчёт показателя Monetary на основе суммарных затрат на покупки по каждому клиенту
dfMonetary = dfWithTotalCostAndDateDif.groupBy("CustomerID").agg(sum("TotalCost").alias('Monetary'))\
    .orderBy("CustomerID")
# Определение максимального, минимального и среднего значения Monetary (позволит приближенно оценить распределение признака)
dfMonetaryMaxMin = dfMonetary.select(round(max("Monetary")).alias("maxMon"),\
                                     round(min("Monetary")).alias("minMon"),\
                                     round(avg("Monetary")).alias("avgMon"))
maxMon = dfMonetaryMaxMin.collect()[0].__getitem__('maxMon')
minMon = dfMonetaryMaxMin.collect()[0].__getitem__('minMon')
avgMon = dfMonetaryMaxMin.collect()[0].__getitem__('avgMon')
# Добавление колонки для указания группы покупателей по показателю Monetary (Стоимость всех покупок клиента)
dfMonetaryGroupABC = dfMonetary\
    .withColumn("MonetaryGroup", \
        when((dfMonetary.Monetary > avgMon)\
              & ((dfMonetary.Monetary - avgMon) >= (maxMon - avgMon)*0.1)\
              , "A")\
        .when((dfMonetary.Monetary > avgMon)\
                | ((dfMonetary.Monetary <= avgMon)\
                    & ((avgMon - dfMonetary.Monetary) >= (avgMon - minMon)*0.6))\
            , "B")\
        .otherwise("C"))                


24/07/07 20:01:24 WARN TaskSetManager: Stage 96 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:27 WARN TaskSetManager: Stage 108 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:30 WARN TaskSetManager: Stage 120 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:33 WARN TaskSetManager: Stage 121 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


Для каждого показателя добавьте стобец с разбиением клиентов на 3 группы. 
  - Допустим, у нас есть 3 клиента, первый клиент последний раз купил товар 
    только в прошлом году, второй клиент в прошлом месяце, а третий клиент 
    на прошлой неделе. Каждый из этих клиентов должен получить различные 
    значения группы для показателя Recency - A, B и С, где А - отражает 
    наибольшую “ценность”, а С - соответственно, наименьшую. 
  - Добавьте итоговый столбец с “суммой” значений групп по каждому показателю 
    и сохраните в отдельный csv-файл Id только тех клиентов, у которых значения 
    групп ААА.

In [ ]:
# Объединение показателей в одну таблицу.
dfRecencyFrequencyMonetary = dfRecencyGroupABC\
    .join(dfFrequencyGroupABC\
          .join(dfMonetaryGroupABC, ["CustomerID"]), ["CustomerID"])
dfRecencyFrequencyMonetary.show(2)

24/07/07 20:01:38 WARN TaskSetManager: Stage 133 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:40 WARN TaskSetManager: Stage 134 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:42 WARN TaskSetManager: Stage 135 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:44 WARN TaskSetManager: Stage 136 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------+------------+---------+--------------+-----------------+-------------+
|CustomerID|Recency|RecencyGroup|Frequency|FrequencyGroup|         Monetary|MonetaryGroup|
+----------+-------+------------+---------+--------------+-----------------+-------------+
|   12346.0|   4919|           C|        1|             B|          77183.6|            A|
|   12347.0|   4961|           C|      182|             B|4309.999999999997|            B|
+----------+-------+------------+---------+--------------+-----------------+-------------+
only showing top 2 rows



In [ ]:
# датафрейм с Id только тех клиентов, у которых значения групп ААА:
dfRFM_AAA = dfRecencyFrequencyMonetary.filter(\
    (dfRecencyFrequencyMonetary.RecencyGroup == "A") &\
    (dfRecencyFrequencyMonetary.FrequencyGroup == "A") &\
    (dfRecencyFrequencyMonetary.MonetaryGroup == "A")
).orderBy("CustomerID")
dfRFM_AAA.show(3)

24/07/07 20:01:48 WARN TaskSetManager: Stage 143 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:50 WARN TaskSetManager: Stage 144 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:52 WARN TaskSetManager: Stage 145 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:01:55 WARN TaskSetManager: Stage 146 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


1


24/07/07 20:01:58 WARN TaskSetManager: Stage 156 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:02:01 WARN TaskSetManager: Stage 157 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:02:03 WARN TaskSetManager: Stage 158 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:02:05 WARN TaskSetManager: Stage 159 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------+------------+---------+--------------+-----------------+-------------+
|CustomerID|Recency|RecencyGroup|Frequency|FrequencyGroup|         Monetary|MonetaryGroup|
+----------+-------+------------+---------+--------------+-----------------+-------------+
|   14096.0|   4695|           A|     5111|             A|65164.78999999966|            A|
+----------+-------+------------+---------+--------------+-----------------+-------------+



  - Добавьте итоговый столбец с “суммой” значений групп по каждому показателю 
    и сохраните в отдельный csv-файл Id только тех клиентов, у которых значения 
    групп ААА.

In [ ]:
dfRFM_AAA.write.options(header=True).mode("overwrite").csv("./csv")

24/07/07 20:02:09 WARN TaskSetManager: Stage 166 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:02:12 WARN TaskSetManager: Stage 167 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:02:14 WARN TaskSetManager: Stage 168 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.
24/07/07 20:02:16 WARN TaskSetManager: Stage 169 contains a task of very large size (33883 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
# spark.stop()